In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('housing.csv')

In [3]:
from sklearn.model_selection import train_test_split

X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
test_data = X_test.join(y_test)
train_data = X_train.join(y_train)

In [6]:
def prepering_data(train_data):
    
    imputer = SimpleImputer(strategy='most_frequent')
    OneHotEnc = OneHotEncoder(handle_unknown='ignore')
    
    train_data_filled = imputer.fit_transform(train_data)
    train_data_finall = pd.DataFrame(train_data_filled, columns=train_data.columns)
    
    cat_X = train_data_finall[['ocean_proximity']]
    cat_X_to_number = OneHotEnc.fit_transform(cat_X)
    
    cat_X_encoded_df = pd.DataFrame(cat_X_to_number.toarray(), columns=OneHotEnc.get_feature_names_out(['ocean_proximity']))
    
    train_data_finall = train_data_finall.drop(columns=['ocean_proximity'])
    finall_data = pd.concat([train_data_finall, cat_X_encoded_df], axis=1)
        
    return finall_data

In [7]:
train_data_finall = prepering_data(train_data)
test_data_finall = prepering_data(test_data)

In [8]:
train_data_finall

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-118.1,34.07,33.0,3437.0,1081.0,3817.0,1042.0,2.25,203700.0,1.0,0.0,0.0,0.0,0.0
1,-118.17,34.03,42.0,882.0,292.0,1248.0,281.0,2.761,120000.0,1.0,0.0,0.0,0.0,0.0
2,-118.13,33.82,44.0,1619.0,280.0,815.0,284.0,5.5449,232200.0,1.0,0.0,0.0,0.0,0.0
3,-123.71,39.88,42.0,1518.0,383.0,656.0,303.0,1.4952,69800.0,0.0,0.0,0.0,0.0,1.0
4,-118.29,33.95,31.0,2839.0,792.0,2390.0,729.0,2.0,109800.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14443,-121.79,37.99,18.0,3646.0,534.0,1651.0,535.0,5.7321,164700.0,0.0,1.0,0.0,0.0,0.0
14444,-122.76,38.44,11.0,2895.0,524.0,1633.0,534.0,4.7283,170200.0,1.0,0.0,0.0,0.0,0.0
14445,-118.26,34.0,37.0,2615.0,697.0,2484.0,630.0,1.9208,103400.0,1.0,0.0,0.0,0.0,0.0
14446,-122.46,37.71,52.0,1642.0,351.0,1138.0,340.0,4.1406,219500.0,0.0,0.0,0.0,0.0,1.0


In [9]:
X_train = train_data_finall.drop(['median_house_value'], axis=1)
y_train = train_data_finall['median_house_value']

X_test = test_data_finall.drop(['median_house_value'], axis=1)
y_test = test_data_finall['median_house_value']

In [21]:
forest = RandomForestRegressor(n_estimators=100, n_jobs=-1)

forest.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [23]:
forest.score(X_test, y_test)

0.8235783420925533